In [24]:
import torchvision 
from torchvision import transforms
from torch.utils.data import DataLoader
import torch as ch
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import cox
from cox.utils import Parameters
import cox.store as store
from cox.readers import CollectionReader
import seaborn as sns
import os
import seaborn as sns
import matplotlib.pyplot as plt

# set environment variable so that stores can create output files
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

TRUNC_CE_CONSTANT_LR = '/home/gridsan/stefanou/cifar-10/resnet-18/trunc_ce_constant'
TRUNC_CE_CYCLIC = '/home/gridsan/stefanou/cifar-10/resnet-18/trunc_ce_cyclic'
TRUNC_CE_COSINE = '/home/gridsan/stefanou/cifar-10/resnet-18/trunc_ce_cosine'
TRUNC_CE_STEP_LR = '/home/gridsan/stefanou/cifar-10/resnet-18/trunc_ce_step_lr'
TRUNC_CE_STEP_LR_REAL = '/home/gridsan/stefanou/cifar-10/resnet-18/trunc_ce_step_lr_real'

CE_CONSTANT_LR = '/home/gridsan/stefanou/cifar-10/resnet-18/ce_constant'
CE_CYCLIC = '/home/gridsan/stefanou/cifar-10/resnet-18/ce_cyclic'
CE_COSINE = '/home/gridsan/stefanou/cifar-10/resnet-18/ce_cosine'
CE_STEP_LR = '/home/gridsan/stefanou/cifar-10/resnet-18/ce_step_lr'
CE_STEP_LR_REAL = '/home/gridsan/stefanou/cifar-10/resnet-18/ce_step_lr_real'

In [ ]:
model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path=BASE_CLASSIFIER_PATH + '15033fc8-d094-461c-b5eb-7fef776f23ec/checkpoint.pt.best')

In [4]:
ce_reader = CollectionReader(CE_CONSTANT_LR)
trunc_reader = CollectionReader(TRUNC_CE_CONSTANT_LR)

100%|██████████| 32/32 [00:00<00:00, 43.18it/s]


In [8]:
ce_logs = ce_reader.df('logs')
trunc_logs = trunc_reader.df('logs')
ce_metadata = ce_reader.df('metadata')
trunc_metadata = trunc_reader.df('metadata')

In [31]:
LR = [1e-1, 1e-2, 1e-3]

In [16]:
ce_exp_id = ce_metadata[ce_metadata['lr'] == LR]['exp_id']
ce_exp_logs = ce_logs[ce_logs.exp_id.isin(ce_exp_id)]

In [21]:
trunc_exp_id = trunc_metadata[trunc_metadata['lr'] == LR]['exp_id']
trunc_exp_logs = trunc_logs[trunc_logs.exp_id.isin(trunc_exp_id)]

In [47]:
def plot(ce_metadata, ce_logs, trunc_metadata, trunc_logs, type_):
    fig = plt.figure()
    fig.set_figheight(10)
    fig.set_figwidth(15)
    for i, lr in enumerate(LR):
        ce_exp_id = ce_metadata[ce_metadata['lr'] == lr]['exp_id']
        ce_exp_logs = ce_logs[ce_logs.exp_id.isin(ce_exp_id)]

        trunc_exp_id = trunc_metadata[trunc_metadata['lr'] == lr]['exp_id']
        trunc_exp_logs = trunc_logs[trunc_logs.exp_id.isin(trunc_exp_id)]

        fig.add_subplot(2, 3, i + 1)
        sns.lineplot(data=ce_exp_logs, x='epoch', y='train_loss', label='ce train loss')
        sns.lineplot(data=ce_exp_logs, x='epoch', y='val_loss', label='ce val loss')

        sns.lineplot(data=trunc_exp_logs, x='epoch', y='train_loss', label='trunc ce train loss')
        sns.lineplot(data=trunc_exp_logs, x='epoch', y='val_loss', label='trunc val loss')
        plt.title('{} LR: {} Loss Curves'.format(type_, lr))

        fig.add_subplot(2, 3, i + 4)

        sns.lineplot(data=ce_exp_logs, x='epoch', y='train_prec1', label='ce train prec1')
        sns.lineplot(data=ce_exp_logs, x='epoch', y='val_prec1', label='ce val prec1')

        sns.lineplot(data=trunc_exp_logs, x='epoch', y='train_prec1', label='trunc ce train prec1')
        sns.lineplot(data=trunc_exp_logs, x='epoch', y='val_prec1', label='trunc val prec1')
        plt.title('{} LR: {} Prec1 Curves'.format(type_, lr))
    plt.tight_layout()
    plt.show()

In [ ]:
plot(ce_metadata, ce_logs, trunc_metadata, trunc_logs, 'Constant')